In [1]:
# Load the data and libraries
import pandas as pd
import numpy as np
import random
from scipy import stats
import matplotlib.pyplot as plt
from functools import reduce
plt.style.use('seaborn-whitegrid')

In [2]:
# Helpful functions

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)


def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]


def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)


def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]


# preserves epsilon-differential privacy
def above_threshold(queries, df, T, epsilon):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon)
    
    for idx, q in enumerate(queries):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q(df) + nu_i >= T_hat:
            return idx
    return -1 # the index of the last element


def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0


def range_query(df, col, a, b):
    return len(df[(df[col] >= a) & (df[col] < b)])


def auto_avg(df, epsilon):
    def create_query(b):
        return lambda df: df.clip(lower=0, upper=b).sum() - df.clip(lower=0, upper=b+1).sum()
    # Construct the stream of queries
    bs = np.logspace(-3,6,1000)
    queries = [create_query(b) for b in bs]
    # Run AboveThreshold, using 1/3 of the privacy budget, to find a good clipping parameter
    epsilon_svt = epsilon / 3
    final_b = bs[above_threshold(queries, df, 0, epsilon_svt)]
    # Compute the noisy sum and noisy count, using 1/3 of the privacy budget for each
    epsilon_sum = epsilon / 3
    epsilon_count = epsilon / 3
    noisy_sum = laplace_mech(df.clip(lower=0, upper=final_b).sum(), final_b, epsilon_sum)
    noisy_count = laplace_mech(len(df), 1, epsilon_count)
    return noisy_sum/noisy_count


def score(df, key, option):
    return len(df[df[key] == option])


def report_noisy_max(df, key, score, sensitivity, epsilon):
    options = df[key].unique().tolist()
    scores = [score(df, key, option) for option in options]
    noisy_scores = [laplace_mech(score, 1, epsilon) for score in scores]
    max_score = np.max(noisy_scores)
    max_score_idx = noisy_scores.index(max_score)
    return options[max_score_idx]

In [3]:
# Import dataset
payroll = pd.read_csv('./Citywide_Payroll_Data__Fiscal_Year_2020.csv', low_memory=False)
# add a Total Paid col, sum of all pay (regular, OT, other)
payroll['Total Paid'] = payroll['Regular Gross Paid'] + payroll['Total OT Paid'] + payroll['Total Other Pay']
payroll

,Fiscal Year,Payroll Number,Agency Name,Last Name,First Name,Mid Init,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay,Total Paid
0,2020,2.0,OFFICE OF THE MAYOR,FULEIHAN,DEAN,A,12/31/2017,MANHATTAN,FIRST DEPUTY MAYOR,ACTIVE,291139.0,per Annum,1820.0,286715.07,0.0,0.00,0.00,286715.07
1,2020,2.0,OFFICE OF THE MAYOR,DE BLASIO,BILL,NaN,01/01/2014,MANHATTAN,MAYOR,ACTIVE,258750.0,per Annum,1820.0,257351.53,0.0,0.00,1500.00,258851.53
2,2020,2.0,OFFICE OF THE MAYOR,ANGLIN,LAURA,L,01/03/2017,MANHATTAN,DEPUTY MAYOR,ACTIVE,251982.0,per Annum,1820.0,248153.39,0.0,0.00,0.00,248153.39
3,2020,2.0,OFFICE OF THE MAYOR,THOMPSON,JAMES,P,03/12/2018,MANHATTAN,DEPUTY MAYOR,ACTIVE,251982.0,per Annum,1820.0,248153.39,0.0,0.00,0.00,248153.39
4,2020,2.0,OFFICE OF THE MAYOR,BEEN,VICKI,L,02/07/2017,MANHATTAN,DEPUTY MAYOR,ACTIVE,251982.0,per Annum,1820.0,243097.64,0.0,0.00,0.00,243097.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590205,2020,996.0,NYC HOUSING AUTHORITY,HOLLOWAY,REESHA,Y,04/18/2019,MANHATTAN,CITY SEASONAL AIDE,CEASED,27375.0,Prorated Annual,-63.0,-607.41,0.0,0.00,-1020.00,-1627.41
590206,2020,996.0,NYC HOUSING AUTHORITY,WATKINS,CHRISTOPHER,T,05/14/2012,BRONX,CARETAKER,CEASED,43941.0,per Annum,-80.0,-3370.82,0.0,0.00,1626.15,-1744.67
590207,2020,996.0,NYC HOUSING AUTHORITY,GREEN,DANIELLE,E,05/14/2012,QUEENS,CARETAKER,CEASED,43941.0,per Annum,-80.0,-1685.41,-5.0,-184.14,0.00,-1869.55
590208,2020,996.0,NYC HOUSING AUTHORITY,SCOTT,PRECIOUS,NaN,03/25/2019,BROOKLYN,CARETAKER,CEASED,31286.0,per Annum,-160.0,-2400.02,0.0,0.00,0.00,-2400.02


In [4]:
# get a differentially private count of number of employees per agency
# privacy cost = epsilon
def dp_agency_sizes(epsilon):
    sensitivity = 1
    keys = payroll.groupby('Agency Name').size().keys().to_list()
    counts = payroll.groupby('Agency Name').size().to_list()
    dp_counts = laplace_mech_vec(counts, sensitivity, epsilon)
    dp_agency_sizes = [(keys[i], dp_counts[i]) for i in range(0, len(keys))]
    return pd.DataFrame(dp_agency_sizes, columns=['Agency Name', 'Agency Size'])

In [13]:
# Select the agencies with between `lower` and `upper` employees
# privacy cost = epsilon
def select_agencies_by_size(lower, upper, epsilon):
    s = dp_agency_sizes(epsilon)
    selected_agencies = s.where(s['Agency Size'] > lower).where(s['Agency Size'] < upper).dropna()
    return selected_agencies

In [33]:
# get a non-differentially private mean column value per agency
def get_mean_value(agencies, key):
    mean_tuple = [(a, payroll[payroll['Agency Name'] == a][key].mean()) for a in agencies['Agency Name']]
    mean = pd.DataFrame(mean_tuple, columns=['Agency Name', 'Mean ' + key])
    return mean

In [34]:
# get a differentially private mean column value per agency
# privacy cost = epsilon
def get_dp_mean_value(agencies, key, epsilon):
    epsilon_i = float(epsilon)/len(agencies)
    dp_mean_tuple = [(a, auto_avg(payroll[payroll['Agency Name'] == a][key], epsilon_i)) for a in agencies['Agency Name']]
    dp_mean = pd.DataFrame(dp_mean_tuple, columns=['Agency Name', 'DP Mean ' + key])
    return dp_mean

In [35]:
# get total pay by agency (dp and non-dp, and % error between the two)
# privacy cost = epsilon
def get_pay_by_agency(selected_agencies, epsilon):
    mean_gross_pay = get_mean_value(selected_agencies, 'Total Paid')
    dp_mean_gross_pay = get_dp_mean_value(selected_agencies, 'Total Paid', epsilon)
    # merge dfs
    dfs = [selected_agencies, mean_gross_pay, dp_mean_gross_pay]
    merged = reduce(lambda left,right: pd.merge(left,right,on=['Agency Name'], how='outer'), dfs)
    merged['% Error Total Paid'] = pct_error(merged['Mean Total Paid'], merged['DP Mean Total Paid'])
    return merged

In [22]:
# privacy cost = epsilon
def get_max_in_col_by_agency(selected_agencies, keys, epsilon):
    columns = { 'Agency Name': selected_agencies['Agency Name'] }
    sensitivity = 1
    epsilon = float(1)/len(keys)
    for key in keys:
        col = 'Most Common {}'.format(key)
        columns[col] = []
        for agency in selected_agencies['Agency Name']:
            noisy_max_value = report_noisy_max(payroll[payroll['Agency Name'] == agency], key, score, sensitivity, epsilon)
            columns[col].append(noisy_max_value)
    return pd.DataFrame(columns)

In [31]:
# if the following functions take too long, modify the min and max agency sizes to be smaller
epsilon = 1
agency_size_min = 500
agency_size_max = 1000
selected_agencies = select_agencies_by_size(agency_size_min, agency_size_max, epsilon)

In [36]:
# get data on total pay by agency
epsilon = 8
pay_by_agency = get_pay_by_agency(selected_agencies, epsilon)

In [37]:
epsilon = 2
keys = ['Work Location Borough', 'Title Description', 'Pay Basis']
max_in_col_by_agency = get_max_in_col_by_agency(selected_agencies, keys, epsilon)

In [38]:
merged = pay_by_agency.merge(max_in_col_by_agency, on='Agency Name')
merged

,Agency Name,Agency Size,Mean Total Paid,DP Mean Total Paid,% Error Total Paid,Most Common Work Location Borough,Most Common Title Description,Most Common Pay Basis
0,ADMIN TRIALS AND HEARINGS,731.302906,47898.380603,46063.310322,3.831174,MANHATTAN,HEARING OFFICER,per Hour
1,BOARD OF ELECTION,983.984095,52825.508438,53991.737346,2.207700,MANHATTAN,TEMPORARY CLERK,per Annum
2,DEPT OF YOUTH & COMM DEV SRVS,701.331681,67037.057518,65540.176303,2.232916,MANHATTAN,ADMINISTRATIVE CONTRACT SPECIALIST,per Annum
3,DISTRICT ATTORNEY QNS COUNTY,866.109772,73217.582994,71473.820240,2.381617,QUEENS,ASSISTANT DISTRICT ATTORNEY,per Annum
4,GUTTMAN COMMUNITY COLLEGE,795.622991,21923.908465,20443.879659,6.750753,MANHATTAN,COLLEGE ASSISTANT,per Hour
5,NYC EMPLOYEES RETIREMENT SYS,511.987194,68654.264882,70690.585512,2.966051,BROOKLYN,ASSOCIATE RETIREMENT BENEFITS EXAMINER,per Annum
6,OFFICE OF THE COMPTROLLER,939.162709,74769.380299,74397.310753,0.497623,MANHATTAN,ACCOUNTANT,per Annum
7,OFFICE OF THE MAYOR,800.565949,68114.407462,67124.251484,1.453666,MANHATTAN,SPECIAL ASSISTANT,per Annum
8,TAXI & LIMOUSINE COMMISSION,881.827242,47157.765607,46599.392413,1.184054,QUEENS,TAXI AND LIMOUSINE INSPECTOR,per Annum
